In [1]:
import pandas as pd
import json
from tqdm.notebook import tqdm
from copy import deepcopy

In [2]:
occ= OccurrenceType.objects.filter(company__name='CCR - Rio Sp')
occ.count()

238

In [13]:
pd.DataFrame().to_excel('CCR.xlsx')

In [3]:
# json_data = occ.form_fields

# Function to serialize the specified columns to JSON-formatted strings with double quotation marks
def serialize_columns(df, columns):
    for column in columns:
        df[column] = df[column].apply(lambda x: json.dumps(x))

In [4]:
# fields = json_data['fields']
def get_df(fields,groups,classe):
    columns = ['occ_kind','occ_type','id', 'dataType', 'apiName', 'displayName', 'selectOptions', 'autofill', 'displayIf', 'requiredIf', 'innerfields']
    df = pd.DataFrame(columns=columns)
    groups = groups.get('groups', [])  # Get the groups if available
    
    
    for field in fields:
        field_data = {column: field.get(column, None) for column in columns}

        # Find the group for the field if groups exist
        if groups:
            for group in groups:
                if field_data['id'] in group['members']:
                    field_data['group'] = group['displayName']
                    break
            else:
                field_data['group'] = None
        else:
            field_data['group'] = None

        df=df.append(field_data, ignore_index=True)
    df['occ_kind'] = classe.occurrence_kind
    df['occ_type'] = classe.name
    return df

In [5]:
def options_df_geral(fields,df,classe):
    # Serialize specified columns to JSON-formatted strings with double quotation marks
    serialize_columns(df, ['selectOptions', 'autofill', 'displayIf', 'requiredIf'])

#     output_file = "output.xlsx"  # Specify the output file path and name

#     df.to_excel(output_file, index=False)  # Export DataFrame to Excel
    options_columns = ['occ_kind','occ_type','displayName', 'apiName', 'options']
    options_df = pd.DataFrame(columns=options_columns)
#     teste = deepcopy(options_df)

    for field in fields:
        if field.get('dataType') == 'select' and 'selectOptions' in field:
            display_name = field.get('displayName')
            api_name = field.get('apiName')
            select_options = field.get('selectOptions').get('options', [])

            options = [option.get('name') for option in select_options]

            options_df=options_df.append({'displayName': display_name, 'apiName': api_name, 'options': options}, ignore_index=True)
    options_df['occ_kind'] = classe.occurrence_kind
    options_df['occ_type'] = classe.name
    return options_df
#     options_df.to_excel("options.xlsx", index=False)

In [6]:
def serialized_options_df(options_df,classe):

    # Convert the 'options' column to a list of dictionaries
    options_list = options_df['options'].tolist()

    # Create a new DataFrame with separate columns for each option
    expanded_df = pd.DataFrame(options_list)

    # Concatenate the original columns with the expanded columns
    options_expanded_df = pd.concat([options_df[['occ_kind','occ_type','displayName', 'apiName']], expanded_df], axis=1)
    
    # Adding eighty new columns with names 'option 1' to 'option eighty'
    for i in range(1, 81):
        
        column_name = f'option {i}'
        expanded_df[column_name] = None
    
    # Export the updated options DataFrame to Excel
    # options_expanded_df.to_excel("options_expanded.xlsx", index=False)
    # options_expanded_df['occ_kind'] = classe.occurrence_kind
    # options_expanded_df['occ_type'] = classe.name

    return options_expanded_df


In [7]:
def get_arrayData(fields,classe):
    array_columns = ['occ_kind','occ_type','displayName', 'apiName', 'options', 'order', 'dataType', 'autofill', 'displayIf', 'requiredIf', 'innerfields']
    array_df = pd.DataFrame(columns=array_columns)
    
    for field in fields:

        if field['dataType'] == 'arrayOfObjects':
            inner_fields = field['innerFields']
            for inner_field in inner_fields:
                options = []
                if inner_field['dataType'] == 'select':
                    try:
                        options = [option['name'] for option in inner_field['selectOptions']['options']]
                    except:
                        pass
                inner_field_data = {
                    'displayName': inner_field['displayName'],
                    'apiName': inner_field['apiName'],
                    'options': options,
                    'order': inner_field['order'],
                    'dataType': inner_field['dataType'],
                    'autofill': inner_field.get('autofill', None),
                    'displayIf': inner_field.get('displayIf', None),
                    'requiredIf': inner_field.get('requiredIf', None),
                    'innerfields': inner_field.get('innerfields', None)
                }
                array_df=array_df.append(inner_field_data, ignore_index=True)
            array_df['occ_kind'] = classe.occurrence_kind
            array_df['occ_type'] = classe.name
            return array_df
            

In [8]:
def options_array(fields):  
    # Serialize specified columns to JSON-formatted strings with double quotation marks
    serialize_columns(array_df, ['options', 'autofill', 'displayIf', 'requiredIf'])

    array_df.to_excel("array.xlsx", index=False)

    array_options_columns = ['occ_kind','occ_type','displayName', 'apiName', 'options']
    array_options_expanded = pd.DataFrame(columns=array_options_columns)

    for field in fields:

        if field.get('dataType') == 'arrayOfObjects':
            inner_fields = field.get('innerFields', [])
            for inner_field in inner_fields:
                if inner_field.get('dataType') == 'select':
                    display_name = inner_field.get('displayName')
                    api_name = inner_field.get('apiName')
                    select_options = inner_field.get('selectOptions', {}).get('options', [])

                    options = [option.get('name') for option in select_options]

                    array_options_expanded=array_options_expanded.append({'displayName': display_name, 'apiName': api_name, 'options': options}, ignore_index=True)
    array_options_expanded['occ_kind'] = classe.occurrence_kind
    array_options_expanded['occ_type'] = classe.name
    return array_options_expanded

In [9]:
# def serialized_array(array_options_expanded):    
#     # Serialize specified columns to JSON-formatted strings with double quotation marks
#     serialize_columns(array_options_expanded, ['displayName', 'apiName', 'options'])

#     # Convert the 'options' column to a list of dictionaries
#     options_list = array_options_expanded['options'].tolist()

#     # Get the maximum number of options in any row
#     max_options = max(len(options) for options in options_list)

#     # Create a new DataFrame with separate columns for each option
#     expanded_df = pd.DataFrame(columns=[f"option_{i+1}" for i in range(max_options)])

#     # Fill the expanded DataFrame with the options
#     for options in options_list:
#         row = {}
#         for i, option in enumerate(options):
#             row[f"option_{i+1}"] = option
#         expanded_df = expanded_df.append(row, ignore_index=True)

In [10]:
def generate_final_df(df_geral, df_final):

    # Concatenar dataFrame da rodada com dataFrame Final
    df_final = df_final.append(df_geral)

    # Concatenar dataFrame da rodada com dataFrame Final
    #options_df_final = options_df_final.append(odf)

    # Concatenar dataFrame da rodada com dataFrame Final
    #options_expanded_df_final = options_expanded_df_final.append(options_expanded_df)
    
    return df_final

In [11]:
def generate_excel_final(df_final):    
    # Concatenate the original columns with the expanded columns
#     array_options_expanded = pd.concat([array_options_expanded[['displayName', 'apiName']], expanded_df], axis=1)

    # Export the updated options DataFrame to Excel
#     array_options_expanded.to_excel("array_options_expanded.xlsx", index=False)

    # Save each DataFrame to separate sheets in a single Excel file
    
    with pd.ExcelWriter('CCR.xlsx',mode='a',if_sheet_exists='replace') as writer:
        df_final.to_excel(writer, sheet_name='df', index=False)
        #options_expanded_df_final.to_excel(writer, sheet_name='options_expanded_df', index=False)
        #options_df_final.to_excel(writer, sheet_name='options_df', index=False)
        #array_df_final.to_excel(writer, sheet_name='array_df', index=False)
#         array_options_expanded.to_excel(writer, sheet_name='array_options_expanded', index=False)

In [13]:
# Criar DataFrame Final
columns = ['occ_kind','occ_type','id', 'dataType', 'apiName', 'displayName', 'selectOptions', 'autofill', 'displayIf', 'requiredIf', 'innerfields']
df_final = pd.DataFrame(columns=columns)

# Criar DataFrame Final para Options
options_columns = ['occ_kind','occ_type','displayName', 'apiName', 'options']
options_df_final = pd.DataFrame(columns=options_columns)

# Criar DataFrame Final para Options Expandida com as mesmas colunas da DataFrame de Options Expandida
# options_expanded_df_final = pd.DataFrame(columns=options_expanded_df.columns)

# Criar DataFrame Final para Array
array_columns = ['occ_kind','occ_type','displayName', 'apiName', 'options', 'order', 'dataType', 'autofill', 'displayIf', 'requiredIf', 'innerfields']
array_df_final = pd.DataFrame(columns=array_columns)

In [14]:
for a in tqdm(occ):
    try:
        fields = a.form_fields['fields']
        df_geral = get_df(fields,a.form_fields,a)
#         odf=options_df_geral(fields,df_geral,a)
#         options_expanded_df=serialized_options_df(odf,a)
#         df_arrayData=get_arrayData(fields,a)
        df_final = generate_final_df(df_geral, df_final)
        
#         print(df_arrayData)
#         arr=options_array(fields,a)
#         serialized_array(arr)

    except Exception as e:
        print(a,e,a.fo)
        pass


generate_excel_final(df_final)

  0%|          | 0/238 [00:00<?, ?it/s]

In [22]:
options_expanded_df.columns

Index(['occ_kind', 'occ_type', 'displayName', 'apiName', 0, 1, 2, 3, 4, 5, 6,
       7, 8],
      dtype='object')

In [35]:
for field in fields:

        if field['dataType'] == 'arrayOfObjects':
            inner_fields = field['innerFields']
            for inner_field in inner_fields:
                options = []
                if inner_field['dataType'] == 'select':
                    try:
                        options = [option['name'] for option in inner_field['selectOptions']['options']]
                    except:
                        pass

In [30]:
for a in tqdm(occ):
    print(a.occurrence_kind,a.name)

  0%|          | 0/3 [00:00<?, ?it/s]

42 Monitoração de Equipamentos de ITS
42 Monitoração de Iluminação
42 Monitoração de Instalações Elétricas de Edificações


In [45]:
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})
df2 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})

In [115]:
df.columns

Index(['A', 'B', 'C'], dtype='object')

In [ ]:
import pandas as pd

# Criando o objeto ExcelWriter
writer = pd.ExcelWriter('output.xlsx', engine='xlsxwriter')

# Escrevendo os DataFrames na mesma planilha
df1.to_excel(writer, sheet_name='Sheet1', startrow=0, startcol=0)
df2.to_excel(writer, sheet_name='Sheet1', startrow=0, startcol=df1.shape[1]+2)

# Salvando o arquivo
writer.save()